# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key


## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [6]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Siavonga,ZM,79.45,light rain,-16.5382,28.7088,Lake Safari Lodge Annex
1,1,Hermanus,ZA,75.43,scattered clouds,-34.4187,19.2345,Aloe guest house
2,2,Karratha,AU,82.54,few clouds,-20.7377,116.8463,Karratha International Hotel
3,5,Chuy,UY,80.33,few clouds,-33.6971,-53.4616,Alerces
4,6,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,Keleti International Resort


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [7]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = .75,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [13]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Buenos Aires"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Buenos Aires"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Junin"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "General Pico"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Mar Del Plata"]


## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [15]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], axis =0)

# Display sample data
itinerary_df

,City ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
160,640,Buenos Aires,AR,87.94,few clouds,-34.6132,-58.3772,Hotel Dazzler Tower
164,654,Junin,AR,89.04,light rain,-34.5838,-60.9433,Central Hotel
9,24,General Pico,AR,84.78,overcast clouds,-35.6566,-63.7568,Euskadi Hotel
34,139,Mar Del Plata,AR,81.82,scattered clouds,-38.0023,-57.5575,Nuevo Ostende
160,640,Buenos Aires,AR,87.94,few clouds,-34.6132,-58.3772,Hotel Dazzler Tower


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [39]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City", "Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,City,Lng,Lat
160,Buenos Aires,-58.3772,-34.6132
164,Junin,-60.9433,-34.5838
9,General Pico,-63.7568,-35.6566
34,Mar Del Plata,-57.5575,-38.0023
160,Buenos Aires,-58.3772,-34.6132


## Use GeoViews to create map that shows the four cities in the itinerary

In [90]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    s = 100,
    color = "City")



In [91]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [50]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [93]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [74]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()


## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [75]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]


## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [77]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [78]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,-58.377566,-34.613202
1,-58.377566,-34.613182
2,-58.377579,-34.612869
3,-58.377196,-34.612831
4,-58.376886,-34.612813


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [79]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "blue")

In [92]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [longitude,latitude]